# HMM

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pylab as plt

## Algoritmo Forward

In [2]:
def forward(y, states, a_0, a, e):
    L = len(y)
 
    fwd = []
    f_prev = {}

    for i, y_i in enumerate(y):
        f_curr = {}
        for st in states:
            if i == 0:
                # caso-base (prob a priori)
                prev_f_sum = a_0[st]
            else:
                prev_f_sum = sum(f_prev[k]*a[k][st] for k in states)
            p = 1                             #Ajuste a partir daqui para estender os sensores
            for k in range(0, len(y_i)):
                p = p*e[st][y_i[k]]         
            f_curr[st] = p*prev_f_sum
 
 
 
        fwd.append(f_curr)
        f_prev = f_curr
 
    posterior = []
    for i in range(L):
        norm_sum = 0
        for st in states:
            norm_sum = norm_sum + fwd[i][st]
        norm_factor = 1.0/norm_sum
        posterior.append({st: fwd[i][st]*norm_factor for st in states})   

    return posterior

### Aplicando o algoritmo Forward para resolver o problema do rastreamento dos animais

 Objetivo: rastrear a posição de um animal em um ambiente triangular através do
som emitido
• Cenário: 03 microfones, cada um localizado em um vértice do ambiente. Os
microfones produzem informação binária ruidosa a cada passo de tempo. O animal
está perto de um dos microfones ou próximo ao meio do triângulo.
• Estados do mundo: {m,c1,c2,c3}
• Efeitos (observações): {mic1, mic2, mic3}
• Modelo de transição: P(ci|ci)=0.8, P(m|ci)=0.1, P(cj≠i|ci)=0.05, P(m|m)=0.7,
P(ci|m)=0.1
• Modelo sensor: P(mici|ci)=0.6, P(micj≠i|ci)=0.1, P(mici|m)=0.4
• Inicialmente: P(m)=P(c1)=P(c2)=P(c2)

In [3]:
states = ('C1', 'C2', 'C3', 'M')
#end_state = 'E'
 
# evidências estendidas.
#utilização de lista no lugar de tupla. Ajuste necessário para permitir mais de um sensor.
observations = [['MIC2_cap', 'MIC3_cap', 'MIC1_nCap'], ['MIC1_cap', 'MIC3_cap', 'MIC2_nCap'], ['MIC3_cap', 'MIC1_nCap', 'MIC2_cap'], ['MIC1_cap', 'MIC2_nCap', 'MIC3_cap']]


# Probabilidades a priori
start_probability = {'C1': 0.25, 'C2': 0.25, 'C3': 0.25, 'M': 0.25}

# Modelo de transição
transition_probability = {
   'C1' : {'C1': 0.8, 'C2': 0.05, 'C3': 0.05, 'M': 0.1},
    'C2' : {'C2': 0.8, 'C1': 0.05, 'C3': 0.05, 'M': 0.1},
    'C3' : {'C3': 0.8, 'C1': 0.05, 'C2': 0.05, 'M': 0.1},
    'M' : {'M': 0.7, 'C1': 0.1,'C2': 0.1, 'C3': 0.1 },
   }

# Modelo sensor ajustado 
emission_probability = {
   'C1' : {'MIC1_cap': 0.6, 'MIC2_cap': 0.1, 'MIC3_cap': 0.1, 'MIC1_nCap': 0.4, 'MIC2_nCap': 0.9, 'MIC3_nCap': 0.9},
   'C2' : {'MIC2_cap': 0.6, 'MIC1_cap': 0.1, 'MIC3_cap': 0.1, 'MIC2_nCap': 0.4, 'MIC1_nCap': 0.9, 'MIC3_nCap': 0.9},
    'C3' : {'MIC3_cap': 0.6, 'MIC2_cap': 0.1, 'MIC1_cap': 0.1, 'MIC3_nCap': 0.4, 'MIC2_nCap': 0.9, 'MIC1_nCap': 0.9},
   'M' : {'MIC1_cap': 0.4, 'MIC2_cap': 0.4, 'MIC3_cap': 0.4, 'MIC1_nCap': 0.6, 'MIC2_nCap': 0.6, 'MIC3_nCap': 0.6}
   }

In [4]:
def example():
    return forward(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability)
 
prob = example()
print (prob)

[{'C1': 0.019230769230769235, 'C2': 0.2596153846153846, 'C3': 0.2596153846153846, 'M': 0.4615384615384616}, {'C1': 0.0837152251316036, 'C2': 0.018978176800286208, 'C3': 0.25620538680386373, 'M': 0.6411012112642465}, {'C1': 0.00863260586220246, 'C2': 0.07747436435997544, 'C3': 0.22063001457264042, 'M': 0.6932630152051816}, {'C1': 0.07181238471447907, 'C2': 0.008333005231651546, 'C3': 0.197095790888381, 'M': 0.7227588191654885}]


# Decodificação (decoding)
Serve para resolver o problema de encontrar a sequência de estados ocultos que melhor explica a sequencia de observacões.

## Algoritmo de Viterbi

In [5]:
def print_dptable(V):
    s = "    " + " ".join(("%7d" % i) for i in range(len(V))) + "\n"
    for y in V[0]:
        s += "%.5s: " % y
        s += " ".join("%.7s" % ("%f" % v[y]) for v in V)
        s += "\n"
    print(s)

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
 
    # Inicializa casos-base (t == 0)
    for y in states:
        p = 1
        for k in range(0, len(obs[0])):
            p = p*emit_p[y][obs[0][k]]
        
        V[0][y] = start_p[y]*p
        path[y] = [y]
 
    # Executa Viterbi para t > 0
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
 
        for y in states:
            p = 1 #Ajuste a partir daqui para permitir utilizar mais de um sensor.
            for k in range(0, len(obs[0])):
                p = p*emit_p[y][obs[t][k]]
            (prob, state) = max((V[t-1][y0] * trans_p[y0][y] * p, y0) for y0 in states)
            V[t][y] = prob
            newpath[y] = path[state] + [y]
 
        # não se faz necessário lembrar de caminhos antigos
        path = newpath
     
    #print_dptable(V)
    (prob, state) = max((V[t][y], y) for y in states)
    
    return (V, path[state])

### Aplicando o algoritmo de Viterbi para resolver o problema do rastreamento dos animais

Objetivo: rastrear a posição de um animal em um ambiente triangular através do
som emitido
• Cenário: 03 microfones, cada um localizado em um vértice do ambiente. Os
microfones produzem informação binária ruidosa a cada passo de tempo. O animal
está perto de um dos microfones ou próximo ao meio do triângulo.
• Estados do mundo: {m,c1,c2,c3}
• Efeitos (observações): {mic1, mic2, mic3}
• Modelo de transição: P(ci|ci)=0.8, P(m|ci)=0.1, P(cj≠i|ci)=0.05, P(m|m)=0.7,
P(ci|m)=0.1
• Modelo sensor: P(mici|ci)=0.6, P(micj≠i|ci)=0.1, P(mici|m)=0.4
• Inicialmente: P(m)=P(c1)=P(c2)=P(c2)

In [6]:
states = ('C1', 'C2', 'C3', 'M')
#end_state = 'E'
 
# evidências estendidas.
#utilização de lista no lugar de tupla. Ajuste necessário para permitir mais de um sensor.
observations = [['MIC2_cap', 'MIC3_cap', 'MIC1_nCap'], ['MIC1_cap', 'MIC3_cap', 'MIC2_nCap'], ['MIC2_cap', 'MIC1_cap', 'MIC3_cap'], ['MIC1_cap', 'MIC2_nCap', 'MIC3_cap']]


# Probabilidades a priori
start_probability = {'C1': 0.25, 'C2': 0.25, 'C3': 0.25, 'M': 0.25}

# Modelo de transição
transition_probability = {
   'C1' : {'C1': 0.8, 'C2': 0.05, 'C3': 0.05, 'M': 0.1},
    'C2' : {'C2': 0.8, 'C1': 0.05, 'C3': 0.05, 'M': 0.1},
    'C3' : {'C3': 0.8, 'C1': 0.05, 'C2': 0.05, 'M': 0.1},
    'M' : {'M': 0.7, 'C1': 0.1,'C2': 0.1, 'C3': 0.1 },
   }

# Modelo sensor ajustado 
emission_probability = {
   'C1' : {'MIC1_cap': 0.6, 'MIC2_cap': 0.1, 'MIC3_cap': 0.1, 'MIC1_nCap': 0.4, 'MIC2_nCap': 0.9, 'MIC3_nCap': 0.9},
   'C2' : {'MIC2_cap': 0.6, 'MIC1_cap': 0.1, 'MIC3_cap': 0.1, 'MIC2_nCap': 0.4, 'MIC1_nCap': 0.9, 'MIC3_nCap': 0.9},
    'C3' : {'MIC3_cap': 0.6, 'MIC2_cap': 0.1, 'MIC1_cap': 0.1, 'MIC3_nCap': 0.4, 'MIC2_nCap': 0.9, 'MIC1_nCap': 0.9},
   'M' : {'MIC1_cap': 0.4, 'MIC2_cap': 0.4, 'MIC3_cap': 0.4, 'MIC1_nCap': 0.6, 'MIC2_nCap': 0.6, 'MIC3_nCap': 0.6}
   }

In [7]:
# input the generated markov model
def example():
    return viterbi(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability)

(v, path_hidden) = example()

print (v)
print ('Sequência mais provável:',path_hidden)

[{'C1': 0.0010000000000000002, 'C2': 0.0135, 'C3': 0.0135, 'M': 0.024000000000000004}, {'C1': 0.00012960000000000003, 'C2': 4.320000000000001e-05, 'C3': 0.0005832, 'M': 0.0016128000000000006}, {'C1': 9.676800000000006e-07, 'C2': 9.676800000000006e-07, 'C3': 2.7993600000000004e-06, 'M': 7.225344000000003e-05}, {'C1': 3.9016857600000024e-07, 'C2': 2.8901376000000022e-08, 'C3': 3.9016857600000024e-07, 'M': 4.855431168000001e-06}]
Sequência mais provável: ['M', 'M', 'M', 'M']
